# STAT 450: Case Studies in Statistics

#  Case study: Relation between mRNA and protein levels 

### Recap: recall the client's claim

Using the median ratio of protein to mRNA levels per gene as a proxy for translation rates, our data show that [...] ***it now becomes possible to predict protein abundance in any given tissue with good accuracy from the measured mRNA abundance***


In this notebook, we continue to consider ways to explore and model how protein levels can depend on mRNA levels.

**Client's model:**
For gene $g$, tissue $t$: 
$$
{\rm{protein}}_{gt}  \approx  \beta_g  ({\rm{ mRNA }}_{gt}) $$

$$\hat{\beta}_g  = {\rm{  median ~of~ the ~12 ~ratios~of~}} 
\frac{{\rm{protein}}_{gt}}{ {\rm{ mRNA }}_{gt}} $$

<h4 style="color:red; font-weight:bold;">Discussion:</h4>Notice that the client's model has no intercept term. Why might the client think that intercept=0 is reasonable?  Think of the biology. (Hint: How is protein made?) 

### Learning objectives for today

* Understand how least squares linear regression models differ from our client's apprach

* Evaluate some of our client's claims using an F test to compare nested models
  - Working toward translating the client's scientific questions into statistical questions

-----------------------

### Load libraries and read in the tidy data file data/tidy_data  

Note that we previously saved `data/tidy_data.csv`, so that we no longer need to re-wrangle the two original data files to get the tidy tibble `dat_tidy`.

In [ ]:
# load libraries
library(tidyverse)
library(broom)
library(modelr)
theme_set(theme_bw()) # (optional - removes grey background that is default for ggplot)

# read in tidy dataset & peek 
gene_data <-  read_csv("data/tidy_data.csv", show_col_types = FALSE)
gene_data |> 
    slice_sample(n = 10)

 Choose 4 genes at random from those genes with **12 complete pairs (no missing data for any of the 12 tissues)** using `filter`. 

In [ ]:
set.seed(450) # keep this the same so we all select the same 4 genes

sample_4_complete_genes <- 
    gene_data |>
    group_by(gene) |>
    mutate(values_available = 12 - sum(is.na(mrna) | is.na(protein))) |>
    filter(values_available == 12) |>
    ungroup() |> 
    filter(gene %in% sample(unique(gene), 4)) |>
    select(-values_available)
   
sample_4_complete_genes |>
    slice_sample(n = 50) |>
    arrange(gene, tissue)

Make 4 scatterplots, one for each gene.

In [ ]:
sample_4_complete_genes |>
    ggplot(aes(x = mrna, y = protein)) +
    geom_point() +
    facet_wrap( ~ gene) +
    xlab("mRNA") + 
    ylab("Protein")  +
    ggtitle("Scatterplot of Protein versus mRNA levels of 4 randomly chosen genes") +
    xlim(0, NA) +
    ylim(0, NA)

**Notes:** 
* The range of protein and mRNA for these four genes varies - it may be helpful to view them each on separate scales if we are not interested in comparing values across genes (hint: add `scales = "free"` to the `facet_wrap()` command)
* As we previously discussed, it may be helpful to view the relationship on the log-scale if the values are particularly right-skewed (common in biology)

<h4 style="color:red; font-weight:bold;">Exercise:</h4> 
Remake the previous set of scatterplots, allowing each have a separate x and y limit:

In [ ]:
### Your code here

-----------------------

## Fit Ordinary Least Squares (OLS) regression models

Let's set aside the client's median by ratio model for the moment and approach the problem with OLS linear models. Why?
* Recall OLS regression is a flexible framework that allows us to perform hypothesis tests regarding effects of interest
* The client's median by ratio model does not come with such utility (e.g. how would we formally test whether or not the data provides evidence that the intercept is zero?)

We'll start off by proposing four different models for consideration. For each gene $g=1,2,...$, for each tissue $t=1,\ldots, 12$:

$
{\rm{A.}}\quad 
{\rm{protein}}_{gt} =  \beta_g ({\rm{ mRNA }}_{gt}) + \epsilon_{gt}, 
\quad \epsilon_{gt} \sim N(0,\sigma_{g}^2)
$


$
{\rm{B.}}\quad 
{\rm{protein}}_{gt} =  \beta_g ({\rm{ mRNA }}_{gt}) + \epsilon_{gt}, 
\quad \epsilon_{gt} \sim N(0,\sigma^2)
$

$
{\rm{C.}}\quad 
{\rm{protein}}_{gt} = \alpha_g + \beta_g ({\rm{ mRNA }}_{gt}) + \epsilon_{gt}, 
\quad \epsilon_{gt} \sim N(0,\sigma_g^2)
$

$
{\rm{D.}}\quad 
{\rm{protein}}_{gt} = \alpha_g + \beta_g ({\rm{ mRNA }}_{gt}) + \epsilon_{gt},
\quad \epsilon_{gt} \sim N(0,\sigma^2)
$

<h4 style="color:red; font-weight:bold;">Discussion:</h4> 

1. What is the difference between models A and B?  

2. What is the difference between models C and D?

3. What is the difference between models B and D?

4. From the scatterplots above and your knowledge of biology, what do you think of models A and B compared to models C and D?


**Overview of the four models**:

Same variances (Models B and D):  
- fit one big model $Y=X \beta + \epsilon$ using one `lm`
- one common estimate of variance

Different variances (Models A and C): 
- fit separate models for each gene: 
  -  low tech: split data set in four and analyze each gene separately
  -  fancy:  (better if more than 2 genes) use gene-by-gene `lm` by using `nest` and `map`
- four estimates of variance (one for each gene)

Comment:  for the models considered here it ends up that 
- models A and B have the same slope estimates
- models C and D have the same slopes and intercept estimates

## Consider Models B and D first

### Fit Model B - intercept=0, same error variance for all genes
$$
{\rm{protein}}_{gt} = \beta_g  ({\rm{ mRNA }}_{gt}) + \epsilon_{gt}, \quad \epsilon_{gt} \sim N(0,\sigma^2)
$$

This can be expressed using the usual $Y=X \beta + \epsilon$ model:  so fit all four genes together with one `lm`, and allow each gene to have it's own slope (interaction terms). However, we need to tell `lm` that we want to force the intercept parameter to be equal to zero (we use `0 +` in the formula call, and use the `:` operator since we don't want to allow additive terms in the model which would give gene-specific intercepts).

In [ ]:
lm_B <- lm(..., data = sample_4_complete_genes)

In [ ]:
tidy(lm_B)

In [ ]:
model.matrix(lm_B)

In [ ]:
summary(lm_B)

### Plot Model B

Here we'll use the fitted values from `lm_B` to make a line with `geom_line`. 
Note that `ggplot2::geom_smooth()` has some built-in capabilities to add in a regression line, but it doesn't have the flexibility to fit a common variance for all models but plot each gene separately with `facet_wrap()`. 

Let's start by adding the predictions to the data frame `sample_4_complete_genes`. 

In [ ]:
sample_4_complete_genes <-  
    sample_4_complete_genes |>
    add_predictions(lm_B, var = 'pred_B')

sample_4_complete_genes |>
    slice_sample(n = 10)

In [ ]:
sample_4_complete_genes |>
    ggplot() +
    geom_point(aes(x = mrna, y = protein)) +
    geom_line(aes(x = mrna, y = pred_B), color = 'red') +    
    facet_wrap( ~ gene, scale = "free") +
    xlab("mRNA") +
    ylab("Protein") +
    xlim(0, NA) +
    ylim(0, NA)

### Fit Model D - gene-specific intercepts, same error variance for all genes
$$
{\rm{protein}}_{gt} =\alpha_g+ \beta_g  ({\rm{ mRNA }}_{gt}) + \epsilon_{gt},
\quad \epsilon_{gt} \sim N(0,\sigma^2)
$$

This can be expressed using the usual $Y=X \beta + \epsilon$ model:  so fit all four genes together with one `lm`, and allow each gene to have it's own slope (additive terms). We also allow each gene to have its own slope (interactive model). To get all additive and interaction terms, we use the `*` operator in the formula call. 

In [ ]:
lm_D <- lm(..., data = sample_4_complete_genes)
tidy(lm_D)

In [ ]:
# A different way to write the same model
lm_D <- lm(..., data = sample_4_complete_genes)
tidy(lm_D)

In [ ]:
model.matrix(lm_D)

In [ ]:
summary(lm_D)

<h4 style="color:red; font-weight:bold;">Discussion:</h4> 

$\hat{Y}= X \hat{\beta}$:  How do the estimated coefficients relate to the slopes and intercepts of the two lines?
Hint:  look at the model matrix.

### Plot Model D

Let's make the four scatterplots again and put the estimated lines for Models B and D. We'll use the fitted values to draw the estimated lines. We'll break this task down step by step in the following exercise.

---

<h4 style="color:red; font-weight:bold;">Exercise:</h4> 

(a) First, add a column named `pred_D` with fitted values from `lm_D` into `sample_4_complete_genes`:

In [ ]:
###  Fill in ... :
sample_4_complete_genes <- 
    sample_4_complete_genes |>
    add_...(..., ... = 'pred_D')

sample_4_complete_genes |> 
    slice_sample(n = 10)

(b) Now make the scatter plot and add the lines. 

In [ ]:
###  Fill in ... :
sample_4_complete_genes |>  
    ggplot() +
    geom_...(aes(...)) +
    geom_...(aes(..., y = pred_D), color = 'blue') +
    geom_...(aes(..., y = pred_B), color = 'red') +
    facet_wrap( ~ gene, scale = "free") +
    xlab("mRNA") +
    ylab("Protein") +
    xlim(0,NA) +
    ylim(0,NA)

---

## Nested Models B and D: F tests

Let's explore whether the data gives us evidence that the intercepts for each gene are different than zero. This is an important consideration for interpreting the models the client has already proposed and whether they seem plausible.

For simplicity of notation, let's consider the two models, B and D, for the simpler case of only two genes (instead of 4).

Re-write each model in standard notation to drop gene subscript and show all parameters estimated by `lm`:

Let 
- $y_i$ be the protein value for observation $i$ ($i = 1, ... , 24$)
- Indicator variable for gene: 
    - $x_{i1} = 1$ if observation $i$ is from gene 2, else 0
- $x_{i2}$ be the mRNA expression value for observation $i$
- when $i=1,...,12$, $g=1$ and when $i=13,...,24$, $g=2$: 
    - $\alpha_g = \alpha_0 + \alpha_1 x_{i1}$
    - $\beta_g = \alpha_2 + \alpha_3 x_{i1}$

Then, plugging these into the B and D models:

D. $
{\rm{protein}}_{gt} =~ \alpha_g ~+ ~\beta_g ({\rm{ mRNA }}_{gt}) + ~\epsilon_{gt}
$ becomes:

$$y_i  =  \alpha_0 ~+~  \alpha_1 x_{i1}  ~+~ \alpha_2 x_{i2} ~+~  \alpha_3 x_{i2} x_{i1}   ~+~ \epsilon_i$$
 
and

B. $
{\rm{protein}}_{gt} =  ~ \beta_g ({\rm{ mRNA }}_{gt})~ + ~\epsilon_{gt}
$ becomes:

$$ Y_i  = \alpha_2 x_{i2} ~+~ \alpha_3x_{i2} x_{i1}  ~+~ \epsilon_i$$


**Important! These models are "nested"**:  Model B is a special case of Model D.  Specifically, Model B is Model D with $\alpha_1=0, \alpha_2=0$.

We would like to carry out the following hypothesist test:  

H$_0$:  $\alpha_1=0, \alpha_2=0$   versus H$_a$: at least one of  $\alpha_1$ or $ \alpha_2$ is non-zero.

Rejecting the null gives us evidence that the model allowing nonzero gene-specific intercepts explains significantly more variation in the data. 

This generalizes up to an arbitrary number of genes - we can test whether the full model that allows nonzero gene-specific intercepts explains significantly more variation in the data compared to the reduced model that fixes them to be zero. So we'll go ahead and test this hypothesis on our 4 gene dataset (so we are actually testing whether four parameters are equal to zero):

In [ ]:
# F-test 
anova(lm_B, lm_D)

**Conclusion?**

## Now consider Models A and C

### Model A: intercept=0, gene-specific error variances
$$
{\rm{protein}}_{gt} = \beta_g  ({\rm{ mRNA }}_{gt}) + \epsilon_{gt},  \quad \epsilon_{gt} \sim N(0,\sigma_g^2)
$$

This requires fitting the data from the each gene separately.  This can be done via
- tidyverse functions `group_by()` and `group_modify()`
- tidyverse functions `nest()` and `map()`
- subsetting data and calling `lm` once for each gene

We'll go with the 1st approach, as it is the simplest. Note that the first two approaches will involve less repetition in the code, but require a bit of a learning curve to get familiar with the particular functions.

`group_modify()` is like `summarize()`, but allows us to pass in a data frame / tibble and return a data / frame tibble as well. 

In [ ]:
sample_4_complete_genes |>
  group_by(gene) |>
  group_modify(~ tidy(lm(..., data = .x))) 

In [ ]:
# Note that Model B and Model A provide exactly the same estimates -- but different variances and std. errors (hence, different p-values)
summary(lm_B)

### Plot Model A
Plot the resulting linear models.  Note that `facet_wrap` + `geom_smooth(method = "lm")` applies model fitting separately for each gene, which is what we did above (so we can take the shortcut here and avoid computing the fitted values ourselves).

In [ ]:
sample_4_complete_genes |> 
    ggplot(aes(x = mrna, y = protein)) + 
    geom_point() + 
    geom_smooth(method = "lm", formula = y ~ 0 + x, se = FALSE) +
    #geom_line(aes(x = mrna, y = pred_B), color = 'red') + # Note that Model B and Model A provide exactly the same estimates -- but different variances and std. errors (hence, different p-values)
    facet_wrap( ~ gene, scale = "free") + 
    xlim(0,NA) +
    ylim(0,NA)

### Model C: gene-specific slopes, intercepts, and error variances

$${\rm{protein}}_{gt} = \alpha_g + \beta_g  ({\rm{ mRNA }}_{gt}) + \epsilon_{gt} 
\quad \epsilon_{gt} \sim N(0,\sigma_{gt}^2)$$

Finally, we'll fit model C in much the same way we did Model A. This is a separate model for each gene, and we allow a non-zero intercept.

In [ ]:
sample_4_complete_genes |>
  group_by(gene) |>
  group_modify(~ tidy(lm(..., data = .x))) 

In [ ]:
# Note that Model C and Model D provide exactly the same estimates -- but different variances and std. errors (hence, different p-values)
summary(lm_D)

**Note** 

* the intercept for the second gene is significantly different from zero

* the slope for the second gene is (possibly) negative! (but not significantly different from 0) 

Plot the results, scatterplots plus lines, with the trick of using fitted values and `geom_abline`.

In [ ]:
sample_4_complete_genes |>
    ggplot(aes(x = mrna, y = protein)) + 
    geom_point() + 
    geom_smooth(method = "lm", formula = y ~ x, se = FALSE) + 
    #geom_line(aes(x = mrna, y = pred_D), color = 'red') + # Note that Model C and Model D provide exactly the same estimates -- but different variances and std. errors (hence, different p-values)
    facet_wrap( ~ gene, scale = "free") + 
    xlim(0, NA) +
    ylim(0, NA)

 <h4 style="color:red; font-weight:bold;">Exercise:</h4>

Consider a new model E:

E.$
{~\rm{protein}}_{gt} =~ \alpha_g ~+ ~\beta  ({\rm{ mRNA }}_{gt}) ~+ ~\epsilon_{gt},
\quad \epsilon_{gt} \sim N(0,\sigma^2)
$
 
This model has gene-specific intercepts but a single common slope for all genes.
 
Fit the model with `lm` and check results with `tidy()`.  

In [ ]:
## Fill in ... 
lm_E <- lm(protein ~ ... + ..., data = sample_4_complete_genes)
model.matrix(...)

**Compare model E to model D:**   

Consider the simpler 2-gene setting.

Recall

D. $
{\rm{protein}}_{gt} =~ \alpha_g ~+ ~\beta_g ({\rm{ mRNA }}_{gt}) + ~\epsilon_{gt}
$ becomes:

$$y_i  =  \alpha_0 ~+~  \alpha_1 x_{i1}  ~+~ \alpha_2 x_{i2} ~+~  \alpha_3 x_{i2} x_{i1}   ~+~ \epsilon_i$$
 
 
<h4 style="color:red; font-weight:bold;">Exercise:</h4>
Write down model E in this form by eliminating the terms that aren't needed in the model D equation.

E.$
{~\rm{protein}}_{gt} =~ \alpha_g ~+ ~\beta  ({\rm{ mRNA }}_{gt}) ~+ ~\epsilon_{gt},
\quad \epsilon_{gt} \sim N(0,\sigma^2)
$ becomes: 
$$ ~y_i  =  ~ .......  ~ +~ \epsilon_i $$

 <h4 style="color:red; font-weight:bold;">Exercise:</h4>
 
 What are the null and alternative hypotheses for testing model E versus model D, in terms of model parameters?
 
 Test your null hypothesis using `anova`.  What do you conclude?

 

In [ ]:
### fill in the code here:
anova(...,...)